# Minimal working example

## MWE LD Clumping

In [88]:
%save MWE_ldclump.sh -f

# Defining bash variables for the different paths,

output=output
bfile=~/MWE0612/genotypes.bed
bgenFile=`echo ~/MWE0612/imputed_genotypes_chr2?.bgen`
sampleFile=~/MWE0612/imputed_genotypes.sample
sumstatsFile=~/MWE0612/output/phenotypes_BMI.fastGWA.snp_stats.gz
unrelated_samples=~/MWE0612/unrelated_samples.txt
ld_sample_size=100
clump_field=P
clump_p1=0.05
clump_p2=1
clump_r2=0.2
clump_kb=1000
clump_annotate=OR
numThreads=20
job_size=1

#Running the workflow for BMI trait

sos dryrun ~/UKBB_GWAS_DEV/workflow/LD_Clumping.ipynb \
    --cwd $output \
    --bfile $bfile \
    --bgenFile $bgenFile \
    --sampleFile $sampleFile \
    --sumstatsFile $sumstatsFile \
    --unrelated_samples $unrelated_samples \
    --ld_sample_size $ld_sample_size \
    --clump_field $clump_field \
    --clump_p1 $clump_p1 \
    --clump_p2 $clump_p2 \
    --clump_r2 $clump_r2 \
    --clump_kb $clump_kb \
    --clump_annotate $clump_annotate \
    --numThreads $numThreads \
    --job_size $job_size
    -q localhost \
    -s build &> 062420-sos-LD-MWE.log

Cell content saved to MWE_ldclump.sh, use option -r to also execute the cell.

## MWE for REGENIE

In [ ]:
cd
sos run ~/project/bioworkflows/GWAS/LMM.ipynb regenie\
    --cwd output \
    --bfile /SAY/dbgapstg/scratch/UKBiobank/MWE/genotypes21_22.bed \
    --sampleFile /SAY/dbgapstg/scratch/UKBiobank/MWE/imputed_genotypes.sample \
    --bgenFile /SAY/dbgapstg/scratch/UKBiobank/MWE/imputed_genotypes_chr*.bgen \
    --phenoFile /SAY/dbgapstg/scratch/UKBiobank/MWE/phenotypes.txt \
    --formatFile /SAY/dbgapstg/scratch/UKBiobank/MWE/regenie_template.yml \
    --phenoCol BMI\
    --covarCol SEX \
    --qCovarCol AGE \
    --numThreads 8 \
    --bsize 1000 \
    --lowmem_prefix /home/dc2325/output \
    --trait \
    --minMAC 4 \
    --bgenMinMAF 0.05 \
    --bgenMinINFO 0.8 \
    --reverse_log_p \
    --p_filter 1\
    --container_lmm /SAY/dbgapstg/scratch/UKBiobank/lmm.sif \
    --container_marp /home/dc2325/scratch60/marp.sif

## MWE for BOLT-LMM

In [8]:
sos run ~/project/bioworkflows/GWAS/LMM.ipynb boltlmm \
    --cwd ~/LMM/boltlmm_output \
    --bfile /SAY/dbgapstg/scratch/UKBiobank/MWE/genotypes.bed \
    --sampleFile /SAY/dbgapstg/scratch/UKBiobank/MWE/imputed_genotypes.sample \
    --bgenFile /SAY/dbgapstg/scratch/UKBiobank/MWE/imputed_genotypes_chr*.bgen \
    --phenoFile /SAY/dbgapstg/scratch/UKBiobank/MWE/phenotypes.txt \
    --formatFile /SAY/dbgapstg/scratch/UKBiobank/MWE/blotLMM_template.yml \
    --LDscoresFile ~/software/BOLT-LMM_v2.3.4/tables/LDSCORE.1000G_EUR.tab.gz \
    --geneticMapFile ~/software/BOLT-LMM_v2.3.4/tables/genetic_map_hg19_withX.txt.gz \
    --phenoCol BMI \
    --covarCol SEX \
    --covarMaxLevels 10 \
    --qCovarCol AGE \
    --numThreads 5 \
    --bgenMinMAF 0.001 \
    --bgenMinINFO 0.1 \
    --lmm-option \
    --p_filter 1 \
    --container_lmm /SAY/dbgapstg/scratch/UKBiobank/lmm.sif \
    --container_marp /home/dc2325/scratch60/marp.sif

rm: invalid option -- 'e'
Try 'rm --help' for more information.
INFO: Running boltlmm_1: Run BOLT analysis
INFO: boltlmm_1 (index=0) is completed.
INFO: boltlmm_1 (index=1) is completed.
INFO: boltlmm_1 output:   /home/hs863/LMM/boltlmm_output/cache/imputed_genotypes_chr21.phenotypes_BMI.boltlmm.snp_stats.gz /home/hs863/LMM/boltlmm_output/cache/imputed_genotypes_chr22.phenotypes_BMI.boltlmm.snp_stats.gz in 2 groups
INFO: Running boltlmm_2: Merge results and log files
INFO: boltlmm_2 is completed.
INFO: boltlmm_2 output:   /home/hs863/LMM/boltlmm_output/phenotypes_BMI.boltlmm.snp_stats.gz /home/hs863/LMM/boltlmm_output/phenotypes_BMI.boltlmm.snp_counts.txt
INFO: Running boltlmm_3: Manhattan and QQ plots using `qqman`
INFO: boltlmm_3 is completed.
INFO: boltlmm_3 output:   /home/hs863/LMM/boltlmm_output/phenotypes_BMI.boltlmm.manhattan.png /home/hs863/LMM/boltlmm_output/phenotypes_BMI.boltlmm.qq.png... (5 items)
INFO: Running boltlmm_4: Generate analysis report: HTML file, and optionally

## MWE for FastGWA

In [1]:
sos run ~/project/bioworkflows/GWAS/LMM.ipynb fastGWA \
    --cwd output \
    --bfile c/genotypes.bed \
    --sampleFile /SAY/dbgapstg/scratch/UKBiobank/MWE/imputed_genotypes.sample \
    --bgenFile /SAY/dbgapstg/scratch/UKBiobank/MWE/imputed_genotypes_chr*.bgen \
    --phenoFile /SAY/dbgapstg/scratch/UKBiobank/MWE/phenotypes.txt \
    --formatFile /SAY/dbgapstg/scratch/UKBiobank/MWE/fastGWA_template.yml \
    --phenoCol BMI \
    --covarCol SEX \
    --qCovarCol AGE \
    --numThreads 1 \
    --bgenMinMAF 0.001 \
    --bgenMinINFO 0.1 \
    --parts 2 \
    --p_filter 1 \
    --container_lmm /SAY/dbgapstg/scratch/UKBiobank/lmm.sif \
    --container_marp /home/dc2325/scratch60/marp.sif 

INFO: Running fastGWA_1: fastGWA mixed model (based on the sparse GRM generated above)
INFO: fastGWA_1 (index=0) is completed.
INFO: fastGWA_1 (index=1) is completed.
INFO: fastGWA_1 output:   /home/hs863/LMM/output/cache/imputed_genotypes_chr21.phenotypes.fastGWA.gz /home/hs863/LMM/output/cache/imputed_genotypes_chr22.phenotypes.fastGWA.gz in 2 groups
INFO: Running fastGWA_2: Merge results and log files
INFO: fastGWA_2 is completed.
INFO: fastGWA_2 output:   /home/hs863/LMM/output/phenotypes_BMI.fastGWA.snp_stats.gz /home/hs863/LMM/output/phenotypes_BMI.fastGWA.snp_counts.txt
INFO: Running fastGWA_3: Manhattan and QQ plots using `qqman`
INFO: fastGWA_3 is completed.
INFO: fastGWA_3 output:   /home/hs863/LMM/output/phenotypes_BMI.fastGWA.manhattan.png /home/hs863/LMM/output/phenotypes_BMI.fastGWA.qq.png... (5 items)
INFO: Running fastGWA_4: Generate analysis report: HTML file, and optionally PPTX file
INFO: fastGWA_4 is completed.
INFO: fastGWA_4 output:   /home/hs863/LMM/output/phenot